TMRM MFI Analysis, Mitochondrial Membrane Potential

Used to process experiments where the MFI of TMRM is measured
Static images
Z-stack images
Timecourse images

Input for all is a list of czi file names within a directory, and the path to that directory
All output a data frame with the image, grouping variable information, and MFI

In [1]:
# cell pose is machine learning model for segmenting cells
from cellpose import models, utils, io

# for clearing jpynb output
from IPython.display import clear_output

# import necessary libraries
from scipy.ndimage import binary_fill_holes
from mpl_toolkits.mplot3d import Axes3D

import math
import skimage
import numpy as np
from czifile import CziFile
import czifile
from skimage import filters, morphology, segmentation
import numpy as np
from skimage.measure import regionprops
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.measure import label, regionprops
from skimage.io import imread
from skimage.measure import label
import czifile as czi
import pandas as pd
from skimage.filters import gaussian, threshold_otsu, sobel, median
from skimage.filters import threshold_otsu
from skimage import exposure
from skimage import morphology
from skimage.measure import label
from skimage.morphology import remove_small_objects
from skimage.morphology import binary_dilation, disk
from os import listdir
from os.path import isfile, join

Static Image Analysis

Writing new stuff down below

In [ ]:
import re
# function to read in files and organize based on name
## should put into tuples with FRAP and MIP files, organized by the image number and condition
def pair_frap_mip_files(folder_path):
    '''
    pair_frap_mip_files()
    Takes a folder path and pairs files containing string FRAP and z-stack into tuples
    Paired based on the first and last segment of the file name (delimited by underscore)
    '''
    images_to_analyze_frap = [f for f in listdir(folder_path) if isfile(join(folder_path, f)) and 'FRAP' in f]
    images_to_analyze_zstack = [f for f in listdir(folder_path) if isfile(join(folder_path, f)) and 'z-stack' in f and 'MIP' not in f]
    # list to store tuples 
    pairs = []
    # pair up frap and z stack files based on condition and image number

    frap_split = [re.split('_', f) for f in images_to_analyze_frap]
    z_split = [re.split('_', f) for f in images_to_analyze_zstack]

    # iterate over all images using an index (compare each file to each file in other group)
    for z_ind in range(len(images_to_analyze_zstack)):
        for f_ind in range(len(images_to_analyze_frap)):
            # if first and last element of the list is the same pair the images up
            if z_split[z_ind][len(z_split[z_ind])-1] == frap_split[f_ind][len(frap_split[f_ind])-1] and z_split[z_ind][0] == frap_split[f_ind][0]:
                # add to the list as a tuple
                pairs.append((images_to_analyze_frap[f_ind], images_to_analyze_zstack[z_ind]))


    return pairs

# for testing
# folder_path_12124 = r'folder_path_here'
# test = pair_frap_mip_files(folder_path)
# print(test)
# print(len(test))

In [ ]:
import xml.etree.ElementTree as ET
import czifile

# file_path = r'C:\Users\bensa\Box\LAB\Lab Folder\EXPERIMENTS\CONFOCAL EXPERIMENTS\MMP_FRAP\101024_dox_noDox_TMRM_FRAP\new settings\Dox_FRAP_01.czi'
#file_path = r'C:\Users\bensa\Box\LAB\Lab Folder\EXPERIMENTS\CONFOCAL EXPERIMENTS\MMP_FRAP\101024_dox_noDox_TMRM_FRAP\new settings\Dox_FRAP_24.czi'

def parse_graphic_annotation_czi(file_path):
    '''
    parse_graphic_annotation_czi(file_path)
    Take in a file path to a czi file
    Uses the package cziFile to read in czi file and then parses metadata xml tree to find the first graphic annotation.
    Returns the data for the annotation's location as a tuple of lists
    '''
    czi2 = czifile.CziFile(file_path)
    # pull out metadata xml tree as a string
    xml_mdata = czi2.metadata()
    #print(xml_mdata)
    # convert to an xml tree object
    #tree_mdata = ET.ElementTree(ET.fromstring(xml_mdata))
    root = ET.fromstring(xml_mdata)
    # get the first location of graphic annotations and the list of elements
    ann_layer = root.find(".//*[@Name='Annotation']")
    elems = ann_layer.find("Elements")

    # check if the annotation is a circle or an ellipse and locate annotation
    # This will favor whichever annotation is listed last, which should be ok since their is only one
    ellipse = True
    for child in elems.findall("*"):
        if child.tag == "Circle":
            ellipse = False
            elem = child
        elif child.tag == "Ellipse":
            ellipse = True
            elem = child

    # organize location information into a list
    geom_data = []
    geom_name = [] # stores the names for each number
    # pull out location information from element
    elem_geom = elem.find("Geometry")
    count = 0
    for child in elem_geom.findall("*"):
        # if circle, grab first four children
        if (ellipse and count < 4):
            geom_data.append(float(child.text))
            geom_name.append(child.tag)
        elif (count < 3): # if ellipse, grab first four children
            geom_data.append(float(child.text))
            geom_name.append(child.tag)
        count+=1
    # return a tuple of lists
    return (geom_data, geom_name)

# for testing 
#test = parse_graphic_annotation_czi(file_path='file_path')
# print(test[0])
# print(test[1])

In [ ]:
# function to process the FRAP data
# frap with cell segmentation
import skimage.measure
import czifile
import matplotlib.patches as mpatches

def identify_inclusion_cell_frap(image_path, graphic):
        '''
        identify_inclusion_cell_frap(file, folder_path, graphic) 
        input: file =  name of czi file, folder_path = path up to but not including the czi file, graphic = tuple output of function to parse annotations
        Outputs a tuple containing the mask of the inclusion contained within the graphic annotation and the cell segmented with cellpose containing that inclusion
        Inclusions and cells are determined using intersection
        Cutoff for a segmented cell to be considered as containing an inclusion is containing over 75% of the inclusion's area
        '''
        # concatonate an image path
        #image_path = fr'{folder_path}\{file}'

        # pull out graphic annotation from metadata
        #graphic = parse_graphic_annotation_czi(image_path)

        # read in the czi file in as an array, and clean off the metadata
        image = czifile.imread(image_path)
        image_squeezed = np.squeeze(image)
        # take just the first frame of the time course image
        green_channel = image_squeezed[0, :, :]

        # create a mask for the inclusions
        contrast_adjusted_green = exposure.adjust_sigmoid(green_channel, cutoff=0.30)
        # normalize the contrast of the inclusions
        contrast_adjusted_green_normalized = (contrast_adjusted_green - contrast_adjusted_green.min()) / (contrast_adjusted_green.max() - contrast_adjusted_green.min())    
        threshold_value_inclusion = 0.25
        # create a binary mask
        inclusion_thresholded = contrast_adjusted_green_normalized > threshold_value_inclusion
        inclusion_thresholded_clean = skimage.morphology.remove_small_objects(inclusion_thresholded, min_size=57)
        inclusion_thresholded_clean = skimage.segmentation.clear_border(inclusion_thresholded_clean)

        # # plot the inclusions against the annotation - can uncomment for troubleshooting
        # if len(graphic[0]) == 4:
        #         # ['CenterX', 'CenterY', 'RadiusX', 'RadiusY']
        #         fig, ax = plt.subplots()
        #         plt.imshow(green_channel)
        #         ellipse = mpatches.Ellipse(xy=(graphic[0][0], graphic[0][1]),
        #                           width=graphic[0][2]*2, height=graphic[0][3]*2, fill=False, edgecolor='lightblue')
        #         ax.add_patch(ellipse)
        #         plt.show()
        # else:
        #         fig, ax = plt.subplots()
        #         plt.imshow(inclusion_thresholded_clean)
        #         ellipse = mpatches.Circle(xy=(graphic[0][0], graphic[0][1]),
        #                                   radius=graphic[0][2]*2, fill=False, edgecolor='blue')
        #         ax.add_patch(ellipse)
        #         plt.show()

        # create an ellipse or circle object based on annotaion
        if len(graphic[0]) == 4:
                rr, cc = skimage.draw.ellipse(r=graphic[0][0], c=graphic[0][1],
                                        r_radius = graphic[0][2], c_radius = graphic[0][3])
        else:
                rr, cc = skimage.draw.disk(center=(graphic[0][0], graphic[0][1]), radius=graphic[0][2])

        # take intersection of filled annotation and inclusion mask to select just inclusion of interest
        annotation = np.zeros(inclusion_thresholded_clean.shape)
        annotation[cc, rr] = 1
        one_inclusion_mask = np.logical_and(inclusion_thresholded_clean, annotation)

        mask_labels = skimage.measure.label(one_inclusion_mask)
        
        # make a copy to apply cell outline contours to
        green_copy = green_channel

        #green_copy = skimage.filters.gaussian(green_copy)
        #green_copy[np.where(one_inclusion_mask)] = skimage.filters.gaussian(green_copy[np.where(one_inclusion_mask)])
        green_copy = skimage.exposure.equalize_adapthist(green_copy)

        green_for_cells = skimage.exposure.equalize_adapthist(green_channel)
        green_for_cells = skimage.exposure.rescale_intensity(green_for_cells)

        model = models.Cellpose(gpu=False, model_type='cyto')
        # use cellpose to segment cells
        cellpose_masks, flows, styles, diams = model.eval(green_for_cells, diameter=140) # 64 is also good
        # for each label - single out that label as binary mask, take the contours, and then draw it on a copy of the original image
        max_label = int(np.max(cellpose_masks))
      
        # keep track of highest intersection and cell label with highest overlap
        max_intersection = -1
        max_overlap_label = -1
        # check to see if the cell contains the inclusion
        for i in range(1, max_label):
                in_range = cellpose_masks == i
                # take intersection
                intersection = np.logical_and(in_range, one_inclusion_mask)
                # if higher than current max (already assigned) and threshold, assign as the max label
                if np.sum(intersection) > max_intersection: # and max_intersection > -1:
                        max_intersection = np.sum(intersection)
                        max_overlap_label = i
        
        # if just one cell is identified, set it as the selected
        if (max_label == 1):
                max_overlap_label = max_label
        
        # mask of just the one cell
        one_cell_mask = cellpose_masks == max_overlap_label

        # plot the inclusions against the annotation
        if len(graphic[0]) == 4:
                # ['CenterX', 'CenterY', 'RadiusX', 'RadiusY']
                fig, axes = plt.subplots(1, 3, figsize=(15, 9))
                axes[0].imshow(green_for_cells)
                ellipse = mpatches.Ellipse(xy=(graphic[0][0], graphic[0][1]),
                                  width=graphic[0][3]*2, height=graphic[0][2]*2, fill=False, edgecolor='lightblue')
                axes[0].add_patch(ellipse)
                axes[0].set_title('original annotation')
                axes[0].axis('off')

                axes[1].imshow(cellpose_masks)
                axes[1].set_title('all cellpose cells')
                axes[1].axis('off')

                axes[2].imshow(one_cell_mask)
                axes[2].set_title('cell mask containing inclusion')
                axes[2].axis('off')
                plt.show()
        else:
                fig, axes = plt.subplots(1, 3, figsize=(15, 9))
                axes[0].imshow(green_channel)
                circle = mpatches.Circle(xy=(graphic[0][0], graphic[0][1]),
                                          radius=graphic[0][2]*2, fill=False, edgecolor='blue')
                axes[0].add_patch(circle)
                axes[0].set_title('original annotation')
                axes[0].axis('off')

                axes[1].imshow(cellpose_masks)
                axes[1].set_title('all cellpose cells')
                axes[1].axis('off')

                axes[2].imshow(one_cell_mask)
                axes[2].set_title('cell mask containing inclusion')
                axes[2].axis('off')
                plt.show()

        # can be uncommented to troubleshoot the cellpose segmentation
        # fig, axes = plt.subplots(1, 2, figsize=(15, 9))
        # # Plot each frame
        # axes[0].imshow(cellpose_masks)
        # axes[0].set_title('all cellpose cells')
        # axes[0].axis('off')

        # axes[1].imshow(one_cell_mask)
        # axes[1].set_title('cell mask containing inclusion')
        # axes[1].axis('off')
        # plt.show()

        # returns the cell mask and inclusion mask in a tuple
        return (one_inclusion_mask, one_cell_mask)


# file_path = r'C:\Users\bensa\Box\LAB\Lab Folder\EXPERIMENTS\CONFOCAL EXPERIMENTS\MMP_FRAP\101024_dox_noDox_TMRM_FRAP\new settings\Dox_FRAP_20.czi'
# test_frap_dox1_1010 = identify_inclusion_cell_frap(file_path, graphic=parse_graphic_annotation_czi(file_path))



In [ ]:
import skimage.transform

# define a file for taking the mean fluorescence intensity from z stacks
# will average and report the mean intensity of each slice within a stack - returning one value for each z stack

# define a function that applies a binary threshold to a single tmrm z stack
def tmrm_mfi_z_stack_one_cell(image_path, cell_mask):
    '''
    tmrm_mfi_z_stack_one_cell(file_name_list, folder_path, cell_mask) - applies a binary threshold to a single one-channel tmrm image.
    image_path: the filepath to the czi file to read
    
    Also prints the raw, adjusted, and masked image - only adds the image based on user input
    returns the mean fluorescent intensity of the mitochondria, determined by the segmentation
    '''
    # read in the czi file in as an array, and clean off the metadata
    #image_path = fr'{folder_path}\{file}'

    image = czifile.imread(image_path)
    image_squeezed = np.squeeze(image)

    # grab the number of slices
    n_slices = image_squeezed.shape[0]

    # initialize a list to accumulate MFIs from each slice
    slice_mean_intensity_list = []

    # iterate over each individual slice
    for current_slice in range(n_slices):
        unprocessed_slice = image_squeezed[current_slice, :, :] # take just the slice

        # grab the channel image from the larger czi file object
        red_slice = unprocessed_slice[:] # grab each pixel of the image array for this slice

        # adjust the contrast of the image via sigmoid
        contrast_adjusted_red = exposure.adjust_sigmoid(red_slice, cutoff=0.25)
        # normalize the contrast of the mitochondria
        #contrast_adjusted_red_normalized = (contrast_adjusted_red - contrast_adjusted_red.min()) / (contrast_adjusted_red.max() - contrast_adjusted_red.min())    
        #threshold_value_mitochondria = 0.08

        contrast_adjusted_red_normalized = (contrast_adjusted_red - contrast_adjusted_red.min()) / (contrast_adjusted_red.max() - contrast_adjusted_red.min())
        threshold_value_mitochondria = np.mean(contrast_adjusted_red_normalized) + (np.std(contrast_adjusted_red_normalized) * 1.85) #1.75 works well
        # create a binary mask
        mitochondria_thresholded = contrast_adjusted_red_normalized > threshold_value_mitochondria

        # if the mask is a different size, resize it to fit
        cell_mask = skimage.transform.resize(cell_mask, red_slice.shape, anti_aliasing=False)

        # remove everything outside of the cell mask
        mitochondria_thresholded_one_cell = np.logical_and(mitochondria_thresholded, cell_mask)

        # plot the raw, contrast adjusted, and thresholded image
        fig, axes = plt.subplots(1, 3, figsize=(15, 9))
        # Plot each frame
        axes[0].imshow(red_slice)
        axes[0].set_title('red channel')
        axes[0].axis('off')

        axes[1].imshow(contrast_adjusted_red)
        axes[1].set_title('contrast adjusted image red channel')
        axes[1].axis('off')

        axes[2].imshow(mitochondria_thresholded_one_cell)
        axes[2].set_title('threshold mitochondria in selected cell')
        axes[2].axis('off')
        plt.show()

        # take the mean fluorescence where it is thresholded
        slice_mean_intensity = np.mean(red_slice[mitochondria_thresholded_one_cell > 0])
        # add to the list for the z stacks
        slice_mean_intensity_list.append(slice_mean_intensity)

    # average the MFI from each slice and return the total MFI for the stack
    z_stack_mfi = np.nanmean(slice_mean_intensity_list)
    return z_stack_mfi
    
# file_path = r'C:\Users\bensa\Box\LAB\Lab Folder\EXPERIMENTS\CONFOCAL EXPERIMENTS\MMP_FRAP\101024_dox_noDox_TMRM_FRAP\new settings\Dox_TMRM_20nM_z-stack_20.czi'
# tmrm_dox1_1010_test = tmrm_mfi_z_stack_one_cell(file_path, test_frap_dox1_1010[1])
# print(tmrm_dox1_1010_test)
    

Wrapper function for the frap and tmrm functions

In [ ]:
# runs the annotation, frap, and tmrm functions
def frap_tmrm_one_cell(file_tuple, folder_path):
    '''
    frap_tmrm_one_cell(file_tuple, folder_path)
    inputs: file = file name, folder_path = path up to file name
    Outputs the mfi and size of the inclusion and cell for the given pair of images
    '''
    frap_image_path = fr'{folder_path}\{file_tuple[0]}'
    tmrm_image_path = fr'{folder_path}\{file_tuple[1]}'
    # run annotation parsing function
    ann_graphic = parse_graphic_annotation_czi(frap_image_path)
    # run the frap function
    masks = identify_inclusion_cell_frap(frap_image_path, ann_graphic)
    # run the tmrm function
    mfi = tmrm_mfi_z_stack_one_cell(tmrm_image_path, masks[1])

    # returns the mfi, area of the inclusion, and area of the cell
    inclusion_area = np.sum(masks[0])
    cell_area = np.sum(masks[1])

    return (mfi, inclusion_area, cell_area)


Wrapper function for all of the previous functions
 - This is so that you can inspect each step and only append to data frame if good

In [ ]:
def process_frap_tmrm_mfi_folder(folder_path):
    '''
    process_frap_tmrm_mfi_folder(folder_path)
    inputs: folder path for images to analyze
    This is a wrapper for each of the functions defined above
    This will output a data frame with the size and MFI of each cell with the inclusion indentified, ready to write to a file
    '''
    # get the file pairs
    pairs = pair_frap_mip_files(folder_path)

    # list to hold list data that is kept by the user
    row_list = []
    # list to hold file names of removed files (frap)
    removed_frap = []
    # list to hold file names of removed files (tmrm)
    removed_tmrm = []

    # run three functions on each pair of tmrm and frap files
    for pair in pairs:
        file_results = frap_tmrm_one_cell(file_tuple=pair, folder_path=folder_path)
        print(pair[0])

        # data_list = [pair[0], pair[1], file_results[0], file_results[1], file_results[2]]
        # row_list.append(data_list)
        # Jiya's chunk for visually inspecting each image as it is printed
        add_df = input("Do you want to add this dataframe? (yes/no): ").lower()
        plt.close('all')
        plt.clf()
        clear_output(wait=True)
        if 'y' in add_df:
         # if segmetnation is good, add the results to be returned at the end
            data_list = [pair[0], pair[1], file_results[0], file_results[1], file_results[2]]
            row_list.append(data_list)
        elif add_df == 'n':
            removed_frap.append(pair[0])
            removed_tmrm.append(pair[1])
        elif add_df == 'stop':
            raise Exception('stop input')

    # print the names of the removed files
    print('REMOVED FRAP FILES')
    for file in removed_frap:
        print(file)
    print('REMOVED TMRM FILES')
    for file in removed_tmrm:
        print(file)

    # data frame format: frap_file, tmrm_file, mfi, inclusion_area, cell_area
    df = pd.DataFrame(row_list, columns = ['frap file', 'tmrm file', 'mfi', 'inclusion area', 'cell area'])
    return df


In [ ]:
#folder_path_101024 = r'C:\Users\bensa\Box\LAB\Lab Folder\EXPERIMENTS\CONFOCAL EXPERIMENTS\MMP_FRAP\101524_dox_no_dox_TMRM_FRAP'
#results_101024 = process_frap_tmrm_mfi_folder(folder_path_101024)

folder_path_112924 = r'C:\Users\tun27424\Box\LAB\Lab Folder\EXPERIMENTS\CONFOCAL EXPERIMENTS\MMP_FRAP\291124_dox_no_dox_TMRM_FRAP\new'
results_112924 = process_frap_tmrm_mfi_folder(folder_path_112924)



In [ ]:
print(results_112924)
import openpyxl

results_112924.to_excel(r'C:\Users\tun27424\Downloads\results_11292024.xlsx', index=False)
results_112924